In [15]:
using DataFrames
using JuMP, Cbc

# 1.8.1 Probabilidade de falha

Dadas duas variáveis independentes $X$ e $Y$ e suas probabilidades $f_X(x)$ e $f_Y(y)$, respectivamente, como calcular $f_Z(z)$ onde $Z = X + Y$?

Calculamos a probabilidade conjunta $f(x, y) = f_X(x)f_Y(y)$.

Seguimos com $F_Z(z) = P(Z \leq z) = P(X + Y \leq z) = \int\int_{R}f(x, y)dA$, com $R$ definida pela região $X + Y \leq z$. Temos, portanto, a probabilidade acumulada de $X + Y \leq z$.

Temos também $F_Z(z) = \int_{-\infty}^z f_Z(t)dt$. Deriva-se $F_Z(z)$ encontrado e tem-se $f_Z(z)$.

# 1.8.2 Implementação para $2$ usinas

Uma implementação possível para duas usinas é calcular a priori as probabilidades para todos os casos de manutenção. 

$f_i$ é uma variável auxiliar que indica que a usina $i$ está em manutenção ($0$ corresponde à manutenção); $P_c$ é a probabilidade da demanda ser suprida em um dado cenário de manutenção; $P_j$ é a probabilidade da demanda ser suprida no cenário de manutenção $j$; $\gamma_j$ é uma variável auxiliar da convolução. Segue:

$$min\ \ G_{disp}$$

$$s.a.\ \ G_{disp} = \sum G_i f_i$$
$$P_c \geq \epsilon$$
$$P_c = \sum_{j = 0} P_j \gamma_j$$
$$\sum_{j = 0} \gamma_j = 1$$

$$\gamma_j \geq 0, \forall j$$
$$\gamma_0 \geq 1 - f_1 - f_2$$
$$\gamma_1 \geq f_1 - f_2$$
$$\gamma_2 \geq f_2 - f_1$$
$$\gamma_3 \geq f_1 + f_2 - 1$$

Sendo assim, como $\sum_{j = 0} \gamma_j = 1$, apenas um $\gamma_j$ será $1$, de modo que $P_c = \sum_{i = 0} P_i \gamma_i$ será igual a apenas um $P_j \gamma_j$. Ou seja, o solver escolhe aquele caso (ou um daqueles casos) em que $P_j \geq \epsilon$.

Sejam duas usinas com capacidades disponíveis $G_1 = 30$ e $G_2 = 40$ e uma demanda $D = 35$.

In [49]:
G8_2 = [30, 40]
D8_2 = 35
p8_2 = [0.5, 0.5]
ϵ8_2 = 0.3 #Probabilidade de falha de cada gerador
nplants8_2 = length(G8_2)
nmonths8_2 = size(D8_2);

In [50]:
tabela8_2 = DataFrames.DataFrame(usinas = [0, 1, 2, [1,2]], probabilidade = [0, 0, 0.5, missing])

,usinas,probabilidade
,Any,Float64⍰
1,0,0.0
2,1,0.0
3,2,0.5
4,"[1, 2]",missing


In [51]:
F_Z8_2 = 0
for X in [0, 1]
    for Y in [0, 1]
        Z = G8_2[1] * X + G8_2[2] * Y
        if Z < D8_2
            F_Z8_2 += (X + (-1)^X * p8_2[1]) * (Y + (-1)^Y * p8_2[2])
        end
    end
end
tabela8_2.probabilidade[4] = 1 - F_Z8_2
tabela8_2

,usinas,probabilidade
,Any,Float64⍰
1,0,0.0
2,1,0.0
3,2,0.5
4,"[1, 2]",0.5


In [52]:
m8_2 = Model(solver = CbcSolver())

@variable(m8_2, f[1:nplants8_2], Bin)
@variable(m8_2, γ[1:nplants8_2^2], Bin)
@variable(m8_2, G_disp)
@variable(m8_2, Pc)

@constraint(m8_2, G_disp == sum(G8_2 .* f))
@constraint(m8_2, Pc >= ϵ8_2)
@constraint(m8_2, Pc == sum(tabela8_2.probabilidade .* γ))
@constraint(m8_2, sum(γ) == 1)

@constraint(m8_2, γ[1] >= 1 - f[1] - f[2])
@constraint(m8_2, γ[2] >= f[1] - f[2])
@constraint(m8_2, γ[3] >= f[2] - f[1])
@constraint(m8_2, γ[4] >= f[1] + f[2] - 1)

@objective(m8_2, Min, G_disp);

In [53]:
@time solve(m8_2)
@show getvalue(f)
@show getvalue(γ)
@show getvalue(G_disp)
@show getvalue(Pc);

  0.002988 seconds (65 allocations: 6.867 KiB)
getvalue(f) = [0.0, 1.0]
getvalue(γ) = [0.0, 0.0, 1.0, 0.0]
getvalue(G_disp) = 40.0
getvalue(Pc) = 0.5


# 1.8.3 Implementação para $n$ usinas

Calcula-se a priori as probabilidades para todos os casos de manutenção. 

$f_i$ é uma variável auxiliar que indica que a usina $i$ está em manutenção ($0$ corresponde à manutenção); $P_c$ é a probabilidade da demanda ser suprida em um dado cenário de manutenção; $P_j$ é a probabilidade da demanda ser suprida no cenário de manutenção $j$; $\gamma_j$ é uma variável auxiliar da convolução. Segue:

$$min\ \ G_{disp}$$

$$s.a.\ \ G_{disp} = \sum G_i f_i$$
$$P_c \geq \epsilon$$
$$P_c = \sum_{j = 0} P_j \gamma_j$$
$$\sum_{j = 0} \gamma_j = 1$$

$$\gamma_j \geq 0, \forall j$$
$$\gamma_j \geq \sum_{k \in K_j} f_k - \sum_{m \in M_j} f_m - |K_j| + 1, \ \ \forall j$$

sendo $K_j$ o conjunto de usinas disponíveis no cenário de manutenção $j$, $M_j$ o conjunto de usinas em manutenção no cenário $j$ e $|K_j|$ o número de usinas disponíveis no cenário $j$.

In [209]:
G8_3 = G8_2
D8_3 = D8_2
ϵ8_3 = 0.5
p8_3 = p8_2
nplants8_3 = length(G8_3);

In [210]:
function prob(G, p, D)
    F_Z = 0
    nplants = length(G)
    for config in 0:(2^nplants - 1) #Para cada configuração de falha
        config_bin = Base.bin(Unsigned(config), nplants, false) #Converte para binário
        X = [parse(Int, config_bin[end - plant + 1]) for plant in 1:nplants] #Converte para array
        Z = sum(G .* X)            
        if Z < D
            F_Z += prod(X .+ (-1).^X .* p)
        end
    end
    return 1 - F_Z
end;

In [211]:
function tabela(G, p, D)
    nplants = length(G)
    tabela = zeros(2^nplants, 2)
    tabela = convert(Array{Any}, tabela)
    
    for config in 0:(2^nplants - 1)
        config_bin = Base.bin(Unsigned(config), nplants, false)
        tabela[config + 1, 1] = config_bin
        G_aux = []
        p_aux = []
        for plant in 1:nplants
            if config_bin[end + 1 - plant] == '1'
                push!(G_aux, G[plant])
                push!(p_aux, p[plant])
            end
        end
        if length(G_aux) == 0 #Se o número de usinas é zero
            probability = 0.0
        else
            probability = prob(G_aux, p_aux, D)
        end
        tabela[config + 1, 2] = probability
    end
    return tabela
end;     

In [212]:
tabela8_3 = tabela(G8_3, p8_3, D8_3);

In [213]:
m8_3 = Model(solver = CbcSolver())

@variable(m8_3, G_disp)
@variable(m8_3, f[1:nplants8_3], Bin)
@variable(m8_3, Pc)
@variable(m8_3, γ[1:2^nplants8_3], Bin)

@constraint(m8_3, G_disp == sum(G8_3 .* f))
@constraint(m8_3, Pc >= ϵ8_3)
@constraint(m8_3, Pc == sum(tabela8_3[:, 2] .* γ))
@constraint(m8_3, sum(γ) == 1)

for j in 0:(2^nplants8_3 - 1)
    j_bin = Base.bin(Unsigned(j), nplants8_3, false)
    j_array = [parse(Int, j_bin[end + 1 - plant]) for plant in 1:nplants8_3]
    @constraint(m8_3, γ[j + 1] >= sum((-1)^(1 + j_array[plant]) * f[plant] for plant in 1:nplants8_3) + 1 - count_ones(j))
end

@objective(m8_3, Min, G_disp);

In [214]:
@time solve(m8_3)
@show getvalue(f)
@show getvalue(γ)
@show getvalue(G_disp)
@show getvalue(Pc);

  0.002703 seconds (65 allocations: 6.867 KiB)
getvalue(f) = [0.0, 1.0]
getvalue(γ) = [0.0, 0.0, 1.0, 0.0]
getvalue(G_disp) = 40.0
getvalue(Pc) = 0.5
